In [1]:
import cv2
import numpy as np

In [7]:

cap = cv2.VideoCapture(0)

frameWidth = 640
frameHeight = 480
cap.set(cv2.
CAP_PROP_FRAME_WIDTH, frameWidth)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frameHeight)


# Determine The colors Of My Pencils 

# ✅ Créer d'abord une fenêtre nommée
cv2.namedWindow("TrackBars")
cv2.resizeWindow("TrackBars",500,500)

## ✅ Ajouter la trackbar à la fenêtre 
def empty(a):
    pass

cv2.createTrackbar("Hue Min","TrackBars",12,179,empty)
cv2.createTrackbar("Hue Max","TrackBars",168,179,empty)
cv2.createTrackbar("Sat Min","TrackBars",0,255,empty)
cv2.createTrackbar("Sat Max","TrackBars",255,255,empty)
cv2.createTrackbar("Val Min","TrackBars",0,255,empty)
cv2.createTrackbar("Val Max","TrackBars",255,255,empty)
while cap.isOpened():
    success, img = cap.read()
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    h_min = cv2.getTrackbarPos("Hue Min","TrackBars")
    h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
    s_min = cv2.getTrackbarPos("Sat Min", "TrackBars")
    s_max = cv2.getTrackbarPos("Sat Max", "TrackBars")
    v_min = cv2.getTrackbarPos("Val Min", "TrackBars")
    v_max = cv2.getTrackbarPos("Val Max", "TrackBars")
    
    lower = np.array([h_min,s_min,v_min])
    upper = np.array([h_max,s_max,v_max])
    mask = cv2.inRange(imgHSV,lower,upper)

    mask = cv2.inRange(imgHSV,lower,upper)
    imgResult = cv2.bitwise_and(img,img,mask=mask)

    cv2.imshow("Mask", mask)
    cv2.imshow("Result", imgResult)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break

In [2]:
myColors =[[60,51,167 ,91,87,204],[22,39,221,33,94,255]] # [h_min,s_min,v_min   ,  h_max,s_max,v_max] FOR Green , Fluo 
myColorValues = [[0,204,0],[15,255,255]] # BGR of Green , Fluo

In [16]:
def findColor(img,myColors,myColorValues): 
    imgHSV =cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    newPoints = []
    for i,color in enumerate(myColors) :
        lower = np.array(color[:3])
        upper = np.array(color[3:])
        mask = cv2.inRange(imgHSV,lower,upper)
        cv2.imshow(f"{str(i)}",mask)
        x,y = get_contours(mask)
        # Draw a circle on the tip of the pen having the same color of the pen 
        #cv2.circle(imgResult,(x,y),30,myColorValues[i],-1)
        if x!= 0 and y!=0 :
            newPoints.append([x,y,i])
    # Return Points 
    return newPoints

def get_contours(img):
    contours, hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    x,y,h,w =0,0,0,0 
    for cnt in contours :
        area = cv2.contourArea(cnt)
        if area > 90:
            print(f"area : {area} ")
            #cv2.drawContours(imgResult, [cnt], -1, (0, 0, 255), 8)
            perimetre = cv2.arcLength(cnt, True)
            print(f"Perimetre : {perimetre}")
            # Detet The corner Points 
            approx= cv2.approxPolyDP(cnt,0.02*perimetre,True)
            print(f"approx : {len(approx)}")
            x,y,w,h = cv2.boundingRect(approx)
            
        # Return The Tipe of the pen 
    return x+w//2 ,y
           


In [8]:
def drawOnCanvas(img,myPoints,myColorValues):
    for point in myPoints :
        cv2.circle(img,(point[0] ,point[1]),10,myColorValues[point[2]],-1)  

In [28]:
cap = cv2.VideoCapture(0)
myPoints = [] # [ x,y,colorId ]
while cap.isOpened() :
    success , img = cap.read()
    cv2.putText(img, "Colors used :", (0,15),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),2)
    cv2.circle(img,(140,10),10,myColorValues[0],-1)
    cv2.circle(img,(160,10),10,myColorValues[1],-1)
    drawOnCanvas(img,myPoints,myColorValues)
    imgResult = img.copy()
    newPoints = findColor(imgResult,myColors,myColorValues)

    if len(newPoints) != 0 :
        for newP in newPoints :
            myPoints.append(newP)
    if len(newPoints) != 0 :
        drawOnCanvas(imgResult,myPoints,myColorValues)


    cv2.imshow("Result", imgResult)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break

area : 177.0 
Perimetre : 129.7401144504547
approx : 10
area : 143.5 
Perimetre : 99.84061968326569
approx : 7
area : 132.5 
Perimetre : 82.18376553058624
approx : 7
area : 123.0 
Perimetre : 65.11269783973694
approx : 10
area : 203.5 
Perimetre : 80.18376553058624
approx : 8
area : 174.0 
Perimetre : 67.94112491607666
approx : 10
area : 208.5 
Perimetre : 114.81118214130402
approx : 8
area : 203.5 
Perimetre : 113.9827550649643
approx : 9
area : 203.5 
Perimetre : 113.9827550649643
approx : 9
area : 203.5 
Perimetre : 113.9827550649643
approx : 9
area : 209.5 
Perimetre : 95.9827550649643
approx : 8
area : 148.5 
Perimetre : 64.18376553058624
approx : 8
area : 143.5 
Perimetre : 104.32590091228485
approx : 8
area : 125.0 
Perimetre : 103.74011445045471
approx : 9
area : 125.0 
Perimetre : 103.74011445045471
approx : 9
area : 125.0 
Perimetre : 103.74011445045471
approx : 9
area : 145.0 
Perimetre : 90.08326029777527
approx : 7
area : 120.0 
Perimetre : 99.74011445045471
approx : 9
are